# Script version 2.3.8 for Machine Learning Core (MLC)

## General settings

In [1]:
import os, datetime
current_directory = os.path.join(os.getcwd(), datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
os.makedirs(current_directory)

from mlc_script_log import *
logger = Logger(current_directory, "")
logging.info("Current directory: " + current_directory)

import external_tools

arff_filename = os.path.join(current_directory, "features.arff")
ucf_filename = os.path.join(current_directory, "MLC_configuration.ucf")
h_filename = os.path.join(current_directory, "MLC_configuration.h")

# private import:
import mlc_configurator
from mlc_configurator import *
import decision_tree_generator
from decision_tree_generator import generateDecisionTree
from decision_tree_generator import generate_subset_of_ARFF
import mlc_test
from mlc_test import *
    
device_name = "LSM6DSOX"    ## list of supported devices available with mlc_configurator.get_devices()

2022-08-02 10:39:06,302 [INFO] Current directory: c:\Users\t-davidhou\Documents\github\STMems_Machine_Learning_Core\tools\mlc_python_script\Script\2022-08-02_10-39-06
2022-08-02 10:39:06,400 [INFO] app version: 9.12.0.0


## Step 1: Load log files, label data, define decision tree results
In this step data are loaded and results are assigned. 
Decision tree results can be configured by associating result names to result values.

In [2]:
result_names  = [] # leave empty here
result_values = [] # leave empty here
datalog_results = [] # leave empty here

# Load class names (folder names) from Logs folder
class_names = os.listdir("../Logs/")
print ("available classes = ", class_names)

# For each class (folder), load all data (files in the folder)
datalogs = []
datalogs_split_by_class = []
for class_name in class_names:
    datalogs_i = os.listdir("../Logs/" + class_name +"/")
    print(class_name, " --> data logs: ", datalogs_i)
    datalogs_split_by_class.append(datalogs_i)
    for datalog_i in datalogs_i:
        datalogs.append("../Logs/" + class_name + "/" + datalog_i)
        datalog_results.append(class_name);
print("All data logs: ", datalogs)

# Assign results and values for decision tree 1:
result_names.append(  class_names )
result_values.append( list(range(0, len(class_names), 1)) )
# Assign results and values for decision tree 2:
result_names.append(  [] )
result_values.append( [] )
# Assign results and values for decision tree 3:
result_names.append(  [] )
result_values.append( [] )
# Assign results and values for decision tree 4:
result_names.append(  [] )
result_values.append( [] )
# Assign results and values for decision tree 5:
result_names.append(  [] )
result_values.append( [] )
# Assign results and values for decision tree 6:
result_names.append(  [] )
result_values.append( [] )
# Assign results and values for decision tree 7:
result_names.append(  [] )
result_values.append( [] )
# Assign results and values for decision tree 8:
result_names.append(  [] )
result_values.append( [] )

dectree_filenames = []
for i in range(0,8): 
    if not result_names[i]:
        break
    else: 
        dectree_filenames.append(os.path.join(current_directory, "dectree{}.txt".format(i+1)))
n_decision_trees = i
logging.info('n_decision_trees = %d' % (n_decision_trees))

2022-08-02 10:39:27,354 [INFO] n_decision_trees = 1


available classes =  ['fastwalk', 'still', 'walk']
fastwalk  --> data logs:  ['log3.txt']
still  --> data logs:  ['log1.txt']
walk  --> data logs:  ['log2.txt']
All data logs:  ['../Logs/fastwalk/log3.txt', '../Logs/still/log1.txt', '../Logs/walk/log2.txt']


## Step 2: Generate ARFF files
ARFF files are CSV files where the first n lines declare data types of CSV columns.
By editing lines below, we decide MLC configuration in terms of:
* filters
* features
All the features are computed within a defined time window ('window_length').
Other device settings like MLC data rate, sensor data rate and full scale are defined here.
The module 'mlc_configurator.py' is used for this step.

In [3]:
# device settings
mlc_odr = "26 Hz"                        ## list of allowed values available with mlc_configurator.get_mlc_odr(device_name)
input_type = "accelerometer+gyroscope"   ## list of allowed values available with mlc_configurator.get_mlc_input_type(device_name)
accelerometer_fs = "2 g"                 ## list of allowed values available with mlc_configurator.get_accelerometer_fs(device_name)
accelerometer_odr = "26 Hz"              ## list of allowed values available with mlc_configurator.get_accelerometer_odr(device_name)
gyroscope_fs = "2000 dps"                ## list of allowed values available with mlc_configurator.get_gyroscope_fs(device_name)
gyroscope_odr = "26 Hz"                  ## list of allowed values available with mlc_configurator.get_gyroscope_odr(device_name)            
window_length = 52                       ## Window length (supported values: from 1 to 255)

# filters configuration
# the list of available filters can be obtained with mlc_configurator.get_filter_names(input_type)
filters_list = []   
#filters_list.append(mlc_configurator.mlc_filter("filter_1", "HP_Acc_XYZ"))
#filters_list.append(mlc_configurator.mlc_filter("filter_2", "BP_Acc_V", coef_a2=-0.5, coef_a3=0.5, coef_gain=1))
#filters_list.append(mlc_configurator.mlc_filter("filter_3", "IIR1_Acc_V", coef_b1=1, coef_b2=0.25, coef_a2=0.75))
#filters_list.append(mlc_configurator.mlc_filter("filter_4", "IIR2_Acc_V", coef_b1=0.59, coef_b2=1.19, coef_b3=0.59, coef_a2=-1.01, coef_a3=0.36))

# features configuration
# full list of available features can be obtained with mlc_configurator.get_feature_names()
# full list of available inputs can be obtained with   mlc_configurator.get_mlc_inputs(device_name, input_type)
features_list = []
features_list.append(mlc_configurator.mlc_feature("MEAN", "Acc_V"))
features_list.append(mlc_configurator.mlc_feature("VARIANCE", "Acc_V"))
features_list.append(mlc_configurator.mlc_feature("ENERGY", "Acc_V"))
features_list.append(mlc_configurator.mlc_feature("PEAK_TO_PEAK", "Acc_V"))
features_list.append(mlc_configurator.mlc_feature("ZERO_CROSSING", "Acc_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("POSITIVE_ZERO_CROSSING", "Acc_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("NEGATIVE_ZERO_CROSSING", "Acc_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("PEAK_DETECTOR", "Acc_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("POSITIVE_PEAK_DETECTOR", "Acc_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("NEGATIVE_PEAK_DETECTOR", "Acc_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("MINIMUM", "Acc_V"))
features_list.append(mlc_configurator.mlc_feature("MAXIMUM", "Acc_V"))
features_list.append(mlc_configurator.mlc_feature("MEAN", "Gyr_V"))
features_list.append(mlc_configurator.mlc_feature("VARIANCE", "Gyr_V"))
features_list.append(mlc_configurator.mlc_feature("ENERGY", "Gyr_V"))
features_list.append(mlc_configurator.mlc_feature("PEAK_TO_PEAK", "Gyr_V"))
features_list.append(mlc_configurator.mlc_feature("ZERO_CROSSING", "Gyr_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("POSITIVE_ZERO_CROSSING", "Gyr_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("NEGATIVE_ZERO_CROSSING", "Gyr_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("PEAK_DETECTOR", "Gyr_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("POSITIVE_PEAK_DETECTOR", "Gyr_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("NEGATIVE_PEAK_DETECTOR", "Gyr_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("MINIMUM", "Gyr_V"))
features_list.append(mlc_configurator.mlc_feature("MAXIMUM", "Gyr_V"))
# Some examples of features on filtered inputs: 
#features_list.append(mlc_configurator.mlc_feature("MEAN","Acc_X_filter_1"))
#features_list.append(mlc_configurator.mlc_feature("ENERGY", "Acc_V_filter_2"))
#features_list.append(mlc_configurator.mlc_feature("ZERO_CROSSING", "Acc_V_filter_3", 0.5))

mlc_configurator.arff_generator( device_name = device_name, 
               datalogs = datalogs, 
               results = datalog_results, 
               mlc_odr = mlc_odr, 
               input_type = input_type, 
               accelerometer_fs = accelerometer_fs, 
               accelerometer_odr = accelerometer_odr, 
               gyroscope_fs = gyroscope_fs, 
               gyroscope_odr = gyroscope_odr, 
               n_decision_trees = n_decision_trees, 
               window_length = window_length, 
               filters_list = filters_list,  
               features_list = features_list, 
               arff_filename = arff_filename, 
               current_directory = current_directory)

2022-08-02 10:39:27,449 [INFO] 
Calling MLC app for features computation and ARFF generation...
2022-08-02 10:39:45,349 [INFO] 
ARFF generated successfully: c:\Users\t-davidhou\Documents\github\STMems_Machine_Learning_Core\tools\mlc_python_script\Script\2022-08-02_10-39-06\features.arff


## Step 3: Decision Tree generation
The module 'decision_tree_generator.py' is used to generate decision trees through sklearn. 

If you have generated the decision tree with external tools, you can skip this section and manually copy the decision tree files in the current directory using: 
* from shutil import copyfile
* source_0 = "PathToYourDecisionTreeFile"
* copyfile(source_0, dectree_filenames[0])

In [4]:
if (n_decision_trees == 1):
    dectree_accuracy, dectree_nodes = decision_tree_generator.generateDecisionTree(
                                                        arff_filename = arff_filename, 
                                                        dectree_filename = dectree_filenames[0])
else:
    for i in range(n_decision_trees) :
        arff_filename_i = arff_filename + str(i+1)
        decision_tree_generator.generate_subset_of_ARFF( arff_filename = arff_filename, 
                                                         arff_subset_filename = arff_filename_i, 
                                                         classes_subset = result_names[i] )
        logging.info("\n# Decision Tree %d:" %(i+1))
        dectree_accuracy, dectree_nodes = decision_tree_generator.generateDecisionTree( 
                                               arff_filename = arff_filename_i, 
                                               dectree_filename = dectree_filenames[i] )


2022-08-02 10:39:45,438 [INFO] Accuracy:1.0
2022-08-02 10:39:45,439 [INFO] 
F11_MIN_on_ACC_V <= 0.578: fastwalk
F11_MIN_on_ACC_V > 0.578
|   F12_MAX_on_ACC_V <= 1.066: still
|   F12_MAX_on_ACC_V > 1.066: walk

Number of Leaves  : 	 3

Size of the Tree : 	 5



## Step 4: Generation of .ucf file
Settings and files from 'Step 2' and 'Step 3' are used in 'Step 4' to generate the MLC configuration in a .ucf file (a text file containing a sequence of register addresses and corresponding values). 
Meta classifiers, which are filters on the outputs of the decision trees, can be configured here. 
The module 'mlc_configurator.py' is used for this step.

In [5]:
# Meta-classifiers
# metaclassifierX_values contains the end counter values of the meta classifier associated to the decision tree 'X'
# 4 end counter values are available in LSM6DSOX (the first 4 values in "metaclasifierX_values")
# 8 end counter values are available in LSM6DSRX/ISM330DHCX (the 8 values in "metaclasifierX_values")
# values allowed for end counters are from 0 to 14
metaclassifier1_values = "0,0,0,0,0,0,0,0"
metaclassifier2_values = "0,0,0,0,0,0,0,0"
metaclassifier3_values = "0,0,0,0,0,0,0,0"
metaclassifier4_values = "0,0,0,0,0,0,0,0"
metaclassifier5_values = "0,0,0,0,0,0,0,0"
metaclassifier6_values = "0,0,0,0,0,0,0,0"
metaclassifier7_values = "0,0,0,0,0,0,0,0"
metaclassifier8_values = "0,0,0,0,0,0,0,0"
metaclassifier_values = [metaclassifier1_values, metaclassifier2_values, metaclassifier3_values, metaclassifier4_values, metaclassifier5_values, metaclassifier6_values, metaclassifier7_values, metaclassifier8_values]

mlc_configurator.ucf_generator( device_name = device_name, 
                                arff_filename = arff_filename, 
                                dectree_filenames = dectree_filenames,
                                result_names = result_names, 
                                result_values = result_values, 
                                metaclassifier_values = metaclassifier_values, 
                                ucf_filename = ucf_filename, 
                                current_directory = current_directory )

2022-08-02 10:39:45,517 [INFO] Classes from ARFF: fastwalk, still, walk
2022-08-02 10:39:45,526 [INFO] 
Calling MLC app for .ucf file generation...
2022-08-02 10:40:06,868 [INFO] 
.ucf file generated successfully: c:\Users\t-davidhou\Documents\github\STMems_Machine_Learning_Core\tools\mlc_python_script\Script\2022-08-02_10-39-06\MLC_configuration.ucf


## (Optional) Generate header file from .ucf file

In [6]:
mlc_configurator.h_generator( ucf_filename = ucf_filename, h_filename = h_filename)
#mlc_configurator.h_generator( ucf_filename="ucftest.ucf", h_filename="ucfthout.h")

2022-08-02 10:40:06,987 [INFO] Header file successfully generated. c:\Users\t-davidhou\Documents\github\STMems_Machine_Learning_Core\tools\mlc_python_script\Script\2022-08-02_10-39-06\MLC_configuration.ucf  ->  c:\Users\t-davidhou\Documents\github\STMems_Machine_Learning_Core\tools\mlc_python_script\Script\2022-08-02_10-39-06\MLC_configuration.h


## (Optional) Testing decision tree
This section allows to test the .arff file on the decision tree.
The .arff file can be generated in Step2 after selecting data patterns (in Step1) and Features (in Step2). 
The module 'mlc_test.py' is used for this step.

In [ ]:
# Testing
testresults_filename = os.path.join(current_directory, "test_results.txt")
mlc_test.test_arff_on_decisiontree( dectree_filename = dectree_filenames[0], 
                                    arff_filename = arff_filename, 
                                    testresults_filename = testresults_filename, 
                                    device_name = device_name )

# show test results:
file_o = open(testresults_filename)   
content = file_o.read()                
logging.info(content)                      
file_o.close()